In [1]:
import torch
import torch.nn as nn
import random

# 一个简单的解码器示例
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        # input: (batch_size)
        embedded = self.embedding(input).unsqueeze(0)  # (1, batch_size, hidden_size)
        output, hidden = self.gru(embedded, hidden)      # output: (1, batch_size, hidden_size)
        output = self.out(output.squeeze(0))             # (batch_size, output_size)
        return output, hidden

def reverse_scheduled_sampling(decoder, target_seq, initial_hidden, rss_rate):
    """
    反向调度采样示例
    参数：
      decoder: 解码器模型
      target_seq: 目标序列，形状为 (seq_len, batch_size)
      initial_hidden: 初始隐藏状态
      rss_rate: 反向调度采样率，取值范围 [0,1]，0表示完全使用真实标签，1表示完全使用模型预测
    返回：
      outputs: 模型生成的输出序列，形状为 (seq_len-1, batch_size, output_size)
    """
    seq_len, batch_size = target_seq.size()
    # 初始输入使用序列开始标记（通常为 <SOS>），这里假设 target_seq[0] 为 <SOS>
    input_token = target_seq[0]  
    hidden = initial_hidden
    outputs = []
    
    # 从 t=1 开始生成序列
    for t in range(1, seq_len):
        output, hidden = decoder(input_token, hidden)
        outputs.append(output)
        # 根据反向调度采样率决定下一个输入
        if random.random() < rss_rate:
            # 使用模型的预测结果作为下一个输入
            input_token = output.argmax(dim=1)
        else:
            # 使用真实标签
            input_token = target_seq[t]
    
    outputs = torch.stack(outputs, dim=0)  # (seq_len-1, batch_size, output_size)
    return outputs

In [3]:
# 示例参数（请根据实际任务设置）
vocab_size = 5000      # 词汇表大小
hidden_size = 256
output_size = vocab_size  # 假设输出词汇分布

# 初始化解码器
decoder = DecoderRNN(input_size=vocab_size, hidden_size=hidden_size, output_size=output_size)

# 假设一个目标序列：seq_len x batch_size
seq_len = 10
batch_size = 32
# 随机生成目标序列，注意 target_seq[0] 通常为 <SOS> 标记
target_seq = torch.randint(0, vocab_size, (seq_len, batch_size))
print("目标序列的形状：", target_seq.shape)

# 初始隐藏状态
initial_hidden = torch.zeros(1, batch_size, hidden_size)

# 设置反向调度采样率，例如 0.3 表示 30% 的概率使用模型预测作为输入
rss_rate = 0.3

# 进行反向调度采样生成序列
outputs = reverse_scheduled_sampling(decoder, target_seq, initial_hidden, rss_rate)
print("生成输出的形状：", outputs.shape)

目标序列的形状： torch.Size([10, 32])
生成输出的形状： torch.Size([9, 32, 5000])
